In [ ]:
import pandas as pd

In [ ]:
colunas_nomes = [
    'ID', 'DATA', 'HORA_MINUTO', 'MUNICIPIO', 'BAIRRO', 
    'ENDERECO', 'ORIGEM_CHAMADO', 'TIPO', 'SUBTIPO', 
    'SEXO', 'IDADE', 'MOTIVO_FINALIZACAO', 'MOTIVO_DESFECHO'
]
df_2025 = pd.read_csv('../data/Dados/samu_2025.csv', header=None, names=colunas_nomes)

df_2025['ID'] = df_2025.index
df_2025.set_index('ID', inplace=True)


df_2025['DATA'] = df_2025['DATA'].astype(str).str.split('T').str[0]

df_2025['DATA'] = pd.to_datetime(df_2025['DATA'], errors='coerce').dt.date

df_2025['HORA_MINUTO'] = df_2025['HORA_MINUTO'].astype(str).str.strip().str[:8]

df_2025['HORA_MINUTO'] = pd.to_datetime(df_2025['HORA_MINUTO'], format='%H:%M:%S', errors='coerce').dt.time

df_2025['IDADE'] = pd.to_numeric(df_2025['IDADE'], errors='coerce')


print(df_2025[['DATA', 'HORA_MINUTO', 'IDADE']].info())
df_2025.head()

In [ ]:
print("Tipo real na coluna DATA:", type(df_2025['DATA'].iloc[0]))

print("Tipo real na coluna HORA:", type(df_2025['HORA_MINUTO'].iloc[0]))

In [ ]:
mediana_idade = df_2025['IDADE'].median()

df_2025['IDADE'].fillna(mediana_idade, inplace=True)

qtd_nulos = df_2025['IDADE'].isnull().sum()

print("Quantidade de valores nulos na coluna IDADE após preenchimento:", qtd_nulos)

In [ ]:
df_2025['MOTIVO_FINALIZACAO'] = df_2025['MOTIVO_FINALIZACAO'].fillna('SEM FINALIZAÇÃO')

print("Quantidade de nulos após tratamento:", df_2025['MOTIVO_FINALIZACAO'].isnull().sum())

df_2025[['MOTIVO_FINALIZACAO']].info()

In [ ]:
df_2025['ENDERECO'] = df_2025['ENDERECO'].fillna('NÃO INFORMADO')

df_2025['ORIGEM_CHAMADO'] = df_2025['ORIGEM_CHAMADO'].fillna('NÃO INFORMADO')

print(df_2025[['ENDERECO', 'ORIGEM_CHAMADO']].isnull().sum())

df_2025.head()

In [ ]:
colunas_restantes = ['SEXO', 'SUBTIPO', 'TIPO', 'MUNICIPIO', 'BAIRRO']

df_2025[colunas_restantes] = df_2025[colunas_restantes].fillna('NÃO INFORMADO')

print("Contagem Final de Nulos:")
print(df_2025.isnull().sum())

df_2025.info()

In [ ]:
colunas_texto = ['MUNICIPIO', 'BAIRRO', 'ENDERECO', 'ORIGEM_CHAMADO', 'TIPO', 'SUBTIPO', 'SEXO', 'MOTIVO_FINALIZACAO', 'MOTIVO_DESFECHO']

for col in colunas_texto:
    df_2025[col] = df_2025[col].astype(str).str.upper().str.strip()

In [ ]:
colunas_texto = ['MUNICIPIO', 'BAIRRO', 'ENDERECO', 'ORIGEM_CHAMADO', 'TIPO', 'SUBTIPO', 'SEXO', 'MOTIVO_FINALIZACAO', 'MOTIVO_DESFECHO']

for col in colunas_texto:
    print(f"\nValores Únicos em {col}")
    valores = sorted(df_2025[col].unique())
    print(valores)

In [ ]:
valores_para_limpar = [
    '93999830', 
    'ANI/ALI', 
    'JOSELENE', 
    'JUSELITA', 
    'MARCILIA', 
    'R MA'
]

df_2025['ORIGEM_CHAMADO'] = df_2025['ORIGEM_CHAMADO'].replace(valores_para_limpar, 'NÃO INFORMADO')

df_2025['ORIGEM_CHAMADO'] = df_2025['ORIGEM_CHAMADO'].replace('ESTAB PR', 'ESTABELECIMENTO PRIVADO')
df_2025['ORIGEM_CHAMADO'] = df_2025['ORIGEM_CHAMADO'].replace('ESTAB PU', 'ESTABELECIMENTO PUBLICO')

In [ ]:
nulos_finalizacao = df_2025['MOTIVO_FINALIZACAO'].isnull().sum()
print("Quantidade de valores nulos na coluna MOTIVO_FINALIZACAO após todas as correções:", nulos_finalizacao)

In [ ]:
colunas_checagem = ['DATA', 'HORA_MINUTO', 'MUNICIPIO', 'BAIRRO', 
    'ENDERECO', 'ORIGEM_CHAMADO', 'TIPO', 'SUBTIPO', 
    'SEXO', 'IDADE', 'MOTIVO_FINALIZACAO', 'MOTIVO_DESFECHO']


qtd_antes = len(df_2025)
df_2025.drop_duplicates(subset=colunas_checagem, keep='first', inplace=True)
qtd_depois = len(df_2025)

print(f"Linhas antes: {qtd_antes}")
print(f"Linhas depois: {qtd_depois}")

In [ ]:
mapa_dias = {
    'Monday': 'SEGUNDA-FEIRA', 'Tuesday': 'TERCA-FEIRA', 'Wednesday': 'QUARTA-FEIRA',
    'Thursday': 'QUINTA-FEIRA', 'Friday': 'SEXTA-FEIRA', 'Saturday': 'SABADO', 'Sunday': 'DOMINGO'
}

df_2025['DATA'] = pd.to_datetime(df_2025['DATA'])
df_2025['DIA_SEMANA'] = df_2025['DATA'].dt.day_name().map(mapa_dias)


def definir_turno(hora_minuto):
    try:
        hora = int(str(hora_minuto)[:2])
        
        if 6 <= hora < 12:
            return 'MANHA'
        elif 12 <= hora < 18:
            return 'TARDE'
        elif 18 <= hora <= 23:
            return 'NOITE'
        else:
            return 'MADRUGADA'
    except:
        return 'NAO INFORMADO'

df_2025['TURNO'] = df_2025['HORA_MINUTO'].apply(definir_turno)

print("Novas colunas geradas:")
df_2025[['DATA', 'DIA_SEMANA', 'HORA_MINUTO', 'TURNO']].head()